In [1]:
import pandas as pd
from tqdm import tqdm
import re
from vncorenlp import VnCoreNLP

tqdm.pandas()

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

In [5]:
product = pd.read_csv('../data/clean/hasaki_comments_cleaned.csv')

print(product[product['Mã sản phẩm'] == 422200355])

      Mã sản phẩm                 Ngày      Tên khách hàng  Số sao đánh giá  \
7766    422200355  2024-05-20 16:10:00                 vân              5.0   
7767    422200355  2024-04-28 12:29:00            Mỹ Phụng              5.0   
7768    422200355  2024-03-11 19:12:00  Trần Ngọc Bảo Trân              2.0   
7769    422200355  2024-03-05 08:51:00              LE HOA              4.0   
7770    422200355  2024-01-13 12:01:00                Ngọc              5.0   
...           ...                  ...                 ...              ...   
7907    422200355  2023-02-11 12:07:00         Dung Nguyễn              5.0   
7908    422200355  2021-11-10 11:20:00          Vy Xuân Vũ              5.0   
7909    422200355  2021-06-22 11:42:00   Bùi Thị Kim Cương              5.0   
7910    422200355  2020-11-04 08:46:00    Nguyễn Xuân Thảo              3.0   
7911    422200355  2020-09-30 21:46:00      Kim Hong Quách              5.0   

                                                   

In [8]:
product[product['Mã sản phẩm'] == 422200355].to_csv('../422200355_comments.csv', encoding='utf-8-sig', index=False)

In [9]:
df = pd.read_csv('../422200355_comments.csv')

df.head()

,Mã sản phẩm,Ngày,Tên khách hàng,Số sao đánh giá,Nội dung đánh giá
0,422200355,2024-05-20 16:10:00,vân,5.0,"Sản phẩm có mùi nhẹ nhàng, thấm không bếch dính"
1,422200355,2024-04-28 12:29:00,Mỹ Phụng,5.0,"Mình sử dụng kết hợp 2 màu sau hơn 1 tháng thì có hiệu quả làm sáng da thật sự nhưng về phần thu nhỏ lcl thì chưa thấy, thay vào đó thì mình có cảm giác lông mọc mỏng hơn và thời gian mọc dài ra cũng lâu hơn. Sản phẩm rất đáng để mua lại ạ"
2,422200355,2024-03-11 19:12:00,Trần Ngọc Bảo Trân,2.0,"Mờ thâm hay thu nhỏ lỗ chân lông thì chưa thấy do mới dùng, cần thêm thời gian. Nhưng hiệu quả ngăn mùi là không có nha mng ơi, bình thường mình dùng sáp khử mùi rất oki chứ loại serum này ko dành cho người dễ đổ mồ hôi và có mùi đâu ạ"
3,422200355,2024-03-05 08:51:00,LE HOA,4.0,"sao nách tui càng dùng càng thâm vậy nè, thâm hơn lúc không dùng quá chán luôn"
4,422200355,2024-01-13 12:01:00,Ngọc,5.0,"loại này cũng mua đến chai thứ 3 luôn, xài rất thích"


<hr>

In [10]:
vncorenlp_file = r'../absolute/path/to/vncorenlp/VnCoreNLP-1.2.jar'
vncorenlp = VnCoreNLP(vncorenlp_file)

def word_tokenize(sentences):

    sentences_tokenized = vncorenlp.tokenize(sentences)
    sentences_cleaned = sentences_tokenized[0]
    # sentences_cleaned = " ".join(words)
    vncorenlp.close()

    return sentences_cleaned

def preprocess_text(text):
    
    # Chuyển đổi thành chữ thường
    text = text.lower()
    # Loại bỏ dấu câu
    text = re.sub(r'[^\w\s]', ' ', text)
    # Thay thế khoảng trắng liên tiếp bằng một khoảng trắng đơn
    text = re.sub(r'\s+', ' ', text)
    # Thay dấu gạch dưới bằng khoảng trống
    text = text.replace('_',' ')
    # Loại bỏ khoảng trống ở hai đầu
    text = text.strip()

    text = word_tokenize(text)

    text = remove_stopwords(text)

    return text

def remove_stopwords(text):
    
    with open('../assets/vietnamese-stopwords.txt', 'r', encoding='utf-8') as file:
        stop_words = [line.strip() for line in file]
    
    # Loại bỏ các từ dừng
    text = [word for word in text if word not in stop_words]
    
    return text

In [11]:
df['Nội dung đánh giá (Cleaned)'] = df['Nội dung đánh giá'].progress_map(preprocess_text)

  0%|          | 0/146 [00:00<?, ?it/s]

100%|██████████| 146/146 [00:02<00:00, 58.35it/s]


In [12]:
for t in df['Nội dung đánh giá (Cleaned)']:
    print(t)

['sản_phẩm', 'mùi', 'nhẹ_nhàng', 'thấm', 'bếch', 'dính']
['sử_dụng', 'kết_hợp', '2', 'màu', '1', 'hiệu_quả', 'da', 'thật_sự', 'thu', 'lcl', 'thay', 'cảm_giác', 'lông', 'mọc', 'mỏng', 'thời_gian', 'mọc', 'sản_phẩm', 'mua']
['mờ', 'thâm', 'thu', 'lỗ_chân_lông', 'thời_gian', 'hiệu_quả', 'ngăn', 'mùi', 'nha', 'mng', 'bình_thường', 'sáp', 'khử', 'mùi', 'oki', 'serum', 'ko', 'đổ', 'mồ_hôi', 'mùi']
['nách', 'tui', 'thâm', 'nè', 'thâm', 'chán']
['mua', 'chai', '3', 'xài']
['mua', 'chai', '3', 'xài', 'ưng']
['10', 'sản_phẩm', 'chất_lượng', 'lắm']
['sản_phẩm', 'chất_lượng']
['10']
['10d']
['ưng', 'giá', 'rẻ', 'phù_hợp', 'túi_tiền', 'hiệu_quả', 'nha', 'mn']
['10đ']
['10đ']
['10']
['10']
['sản_phẩm', '10', 'nha']
['10đ']
['10đ']
['sản_phẩm', '10', 'ngăn', 'mùi', 'mà_còn', 'thoáng', 'bết', 'dính']
['da', 'mịn', 'hẳn', 'màu_da']
['sản_phẩm', 'ok', 'lắm']
['lắm', 'khô', 'thoáng', 'mùi', 'thơm', 'nhẹ', 'dễ_chịu', 'mua']
['10đ']
['sản_phẩm', 'hợp', 'giá', 'tiền', 'moi', 'mua', 'sài', 'mua', 'tiếp', 'sh

In [13]:
data_words = df['Nội dung đánh giá (Cleaned)'].values.tolist()
len(data_words)

146

In [14]:
import gensim.corpora as corpora

# Create Dictionary
id2word = corpora.Dictionary(data_words)
# Create Corpus
texts = data_words
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)]


In [15]:
from gensim.models import LdaMulticore
from gensim.models import LdaModel
from pprint import pprint

# number of topics
num_topics = 10
# Build LDA model
lda_model = LdaMulticore(corpus=corpus, id2word=id2word,
                     num_topics=num_topics, iterations=400)
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.137*"10" + 0.035*"lắm" + 0.035*"đ" + 0.019*"sản_phẩm" + 0.019*"san" + '
  '0.019*"https" + 0.019*"dove" + 0.019*"kem" + 0.019*"pham" + 0.019*"40ml"'),
 (1,
  '0.056*"thâm" + 0.029*"10đ" + 0.029*"10" + 0.029*"nha" + 0.029*"nè" + '
  '0.029*"chán" + 0.029*"nách" + 0.029*"da" + 0.029*"sản" + 0.029*"tui"'),
 (2,
  '0.093*"sản_phẩm" + 0.071*"mua" + 0.041*"lắm" + 0.027*"mùi" + 0.027*"moi" + '
  '0.018*"10" + 0.018*"da" + 0.018*"nách" + 0.018*"hợp" + 0.018*"nhẹ_nhàng"'),
 (3,
  '0.047*"mồ_hôi" + 0.047*"xài" + 0.032*"ngăn" + 0.032*"mùi" + 0.032*"mua" + '
  '0.032*"k" + 0.032*"3" + 0.032*"chai" + 0.017*"10" + 0.017*"sản_phẩm"'),
 (4,
  '0.123*"sản_phẩm" + 0.056*"mùi" + 0.040*"10" + 0.032*"mua" + 0.028*"lắm" + '
  '0.021*"thơm" + 0.020*"ok" + 0.017*"mồ_hôi" + 0.017*"kem" + 0.017*"khô"'),
 (5,
  '0.032*"10" + 0.032*"sản_phẩm" + 0.032*"mua" + 0.032*"mùi" + 0.032*"sử_dụng" '
  '+ 0.032*"hàng" + 0.032*"hương" + 0.032*"công_dụng" + 0.032*"dễ_chịu" + '
  '0.032*"giao"'),
 (6,
  '0.237*"10đ" 

In [28]:
topics = lda_model.print_topics()

In [25]:
for topic in topics:
    print(topic[1])

0.137*"10" + 0.035*"lắm" + 0.035*"đ" + 0.019*"sản_phẩm" + 0.019*"san" + 0.019*"https" + 0.019*"dove" + 0.019*"kem" + 0.019*"pham" + 0.019*"40ml"
0.056*"thâm" + 0.029*"10đ" + 0.029*"10" + 0.029*"nha" + 0.029*"nè" + 0.029*"chán" + 0.029*"nách" + 0.029*"da" + 0.029*"sản" + 0.029*"tui"
0.093*"sản_phẩm" + 0.071*"mua" + 0.041*"lắm" + 0.027*"mùi" + 0.027*"moi" + 0.018*"10" + 0.018*"da" + 0.018*"nách" + 0.018*"hợp" + 0.018*"nhẹ_nhàng"
0.047*"mồ_hôi" + 0.047*"xài" + 0.032*"ngăn" + 0.032*"mùi" + 0.032*"mua" + 0.032*"k" + 0.032*"3" + 0.032*"chai" + 0.017*"10" + 0.017*"sản_phẩm"
0.123*"sản_phẩm" + 0.056*"mùi" + 0.040*"10" + 0.032*"mua" + 0.028*"lắm" + 0.021*"thơm" + 0.020*"ok" + 0.017*"mồ_hôi" + 0.017*"kem" + 0.017*"khô"
0.032*"10" + 0.032*"sản_phẩm" + 0.032*"mua" + 0.032*"mùi" + 0.032*"sử_dụng" + 0.032*"hàng" + 0.032*"hương" + 0.032*"công_dụng" + 0.032*"dễ_chịu" + 0.032*"giao"
0.237*"10đ" + 0.036*"mùi" + 0.027*"ngăn" + 0.027*"serum" + 0.027*"mua" + 0.027*"ok" + 0.018*"sản_phẩm" + 0.018*"mồ_hôi" +

In [35]:
list_topics = [topic[1] for topic in test]

In [36]:
list_topics

['0.026*"phone" + 0.021*"price" + 0.019*"i" + 0.019*"camera" + 0.018*"bad" + 0.016*"evil" + 0.016*"mobile" + 0.015*"great" + 0.012*"redmi" + 0.011*"performance"',
 '0.047*"phone" + 0.043*"i" + 0.039*"good" + 0.021*"camera" + 0.016*"battery" + 0.015*"awesome" + 0.013*"redmi" + 0.012*"flipkart" + 0.012*"mobile" + 0.012*"price"',
 '0.042*"phone" + 0.035*"i" + 0.030*"good" + 0.029*"camera" + 0.021*"mobile" + 0.019*"battery" + 0.018*"bad" + 0.016*"price" + 0.013*"evil" + 0.013*"product"',
 '0.039*"phone" + 0.020*"excellent" + 0.019*"camera" + 0.019*"price" + 0.015*"money" + 0.015*"awesome" + 0.015*"bad" + 0.014*"battery" + 0.013*"face" + 0.013*"i"',
 '0.040*"camera" + 0.039*"phone" + 0.032*"good" + 0.022*"battery" + 0.016*"mobile" + 0.016*"performance" + 0.014*"price" + 0.014*"display" + 0.013*"i" + 0.013*"evil"',
 '0.046*"phone" + 0.028*"camera" + 0.026*"i" + 0.022*"bad" + 0.019*"evil" + 0.014*"performance" + 0.013*"price" + 0.013*"great" + 0.012*"display" + 0.012*"good"',
 '0.034*"phone" 

In [34]:
test = [(0,
  '0.026*"phone" + 0.021*"price" + 0.019*"i" + 0.019*"camera" + 0.018*"bad" + '
  '0.016*"evil" + 0.016*"mobile" + 0.015*"great" + 0.012*"redmi" + '
  '0.011*"performance"'),
 (1,
  '0.047*"phone" + 0.043*"i" + 0.039*"good" + 0.021*"camera" + 0.016*"battery" '
  '+ 0.015*"awesome" + 0.013*"redmi" + 0.012*"flipkart" + 0.012*"mobile" + '
  '0.012*"price"'),
 (2,
  '0.042*"phone" + 0.035*"i" + 0.030*"good" + 0.029*"camera" + 0.021*"mobile" '
  '+ 0.019*"battery" + 0.018*"bad" + 0.016*"price" + 0.013*"evil" + '
  '0.013*"product"'),
 (3,
  '0.039*"phone" + 0.020*"excellent" + 0.019*"camera" + 0.019*"price" + '
  '0.015*"money" + 0.015*"awesome" + 0.015*"bad" + 0.014*"battery" + '
  '0.013*"face" + 0.013*"i"'),
 (4,
  '0.040*"camera" + 0.039*"phone" + 0.032*"good" + 0.022*"battery" + '
  '0.016*"mobile" + 0.016*"performance" + 0.014*"price" + 0.014*"display" + '
  '0.013*"i" + 0.013*"evil"'),
 (5,
  '0.046*"phone" + 0.028*"camera" + 0.026*"i" + 0.022*"bad" + 0.019*"evil" + '
  '0.014*"performance" + 0.013*"price" + 0.013*"great" + 0.012*"display" + '
  '0.012*"good"'),
 (6,
  '0.034*"phone" + 0.032*"bad" + 0.026*"camera" + 0.024*"good" + 0.021*"face" '
  '+ 0.019*"evil" + 0.016*"performance" + 0.015*"smile" + 0.015*"mobile" + '
  '0.013*"battery"'),
 (7,
  '0.029*"bad" + 0.025*"product" + 0.022*"phone" + 0.018*"battery" + 0.017*"i" '
  '+ 0.017*"camera" + 0.015*"money" + 0.013*"good" + 0.013*"redmi" + '
  '0.010*"note"'),
 (8,
  '0.063*"good" + 0.030*"camera" + 0.026*"phone" + 0.025*"i" + 0.021*"battery" '
  '+ 0.015*"evil" + 0.015*"performance" + 0.012*"display" + 0.011*"money" + '
  '0.011*"product"'),
 (9,
  '0.044*"i" + 0.038*"phone" + 0.025*"good" + 0.021*"camera" + 0.018*"product" '
  '+ 0.016*"flipkart" + 0.016*"battery" + 0.016*"quality" + 0.015*"evil" + '
  '0.014*"thank"')]

In [53]:
topics = [
    '0.033*"sản_phẩm" + 0.029*"che" + 0.027*"khuyết_điểm" + 0.022*"mắt" + 0.020*"bút" + 0.016*"maybelline" + 0.013*"thâm" + 0.010*"tự_tin" + 0.010*"phù_hợp" + 0.010*"che_phủ"',
    '0.026*"kem" + 0.022*"sản_phẩm" + 0.020*"mắt" + 0.020*"che" + 0.018*"bút" + 0.017*"khuyết_điểm" + 0.015*"cushion" + 0.015*"thâm" + 0.011*"đầu" + 0.011*"da"',
    '0.025*"che" + 0.025*"sản_phẩm" + 0.025*"sử_dụng" + 0.021*"mắt" + 0.020*"giới_thiệu" + 0.018*"kem" + 0.018*"bạn_bè" + 0.018*"khuyết_điểm" + 0.016*"thâm" + 0.014*"bút"',
    '0.038*"che" + 0.036*"kem" + 0.033*"khuyết_điểm" + 0.025*"sản_phẩm" + 0.024*"mắt" + 0.022*"da" + 0.016*"thâm" + 0.015*"maybelline" + 0.013*"sử_dụng" + 0.012*"tán"',
    '0.045*"mắt" + 0.039*"che" + 0.030*"khuyết_điểm" + 0.024*"thâm" + 0.016*"da" + 0.015*"sản_phẩm" + 0.015*"tán" + 0.015*"có_thể" + 0.015*"chị_em" + 0.014*"màu"',
    '0.027*"che" + 0.026*"khuyết_điểm" + 0.025*"che_phủ" + 0.025*"kem" + 0.021*"mắt" + 0.019*"thâm" + 0.012*"da" + 0.012*"sản_phẩm" + 0.011*"quầng" + 0.010*"mịn"',
    '0.025*"khuyết_điểm" + 0.019*"i" + 0.015*"che" + 0.014*"thâm" + 0.014*"t" + 0.012*"it" + 0.011*"tán" + 0.008*"mắt" + 0.007*"da" + 0.007*"hết_sức"',
    '0.045*"che" + 0.036*"khuyết_điểm" + 0.028*"kem" + 0.025*"thâm" + 0.021*"da" + 0.021*"sản_phẩm" + 0.018*"mắt" + 0.017*"màu" + 0.016*"bút" + 0.016*"quầng"',
    '0.028*"che" + 0.024*"kem" + 0.022*"khuyết_điểm" + 0.018*"bút" + 0.017*"da" + 0.013*"mắt" + 0.012*"sử_dụng" + 0.012*"maybelline" + 0.011*"sản_phẩm" + 0.011*"màu"',
    '0.024*"che" + 0.022*"khuyết_điểm" + 0.019*"thâm" + 0.016*"da" + 0.012*"quầng" + 0.012*"mắt" + 0.011*"mua" + 0.011*"sử_dụng" + 0.010*"1" + 0.010*"kem"'
]

In [37]:
from collections import Counter
import re

# Tách các từ khóa và đếm tần suất
def extract_keywords(topics):
    keywords = []
    for topic in topics:
        words = re.findall(r'"\w+"', topic)  # Tìm tất cả các từ khóa được đặt trong dấu ngoặc kép
        keywords.extend(words)
    return Counter(keywords)

# Tổng hợp từ khóa
keywords = extract_keywords(list_topics)

# Hiển thị từ khóa và tần suất
print(keywords)

Counter({'"phone"': 10, '"camera"': 10, '"i"': 9, '"good"': 8, '"battery"': 8, '"evil"': 7, '"price"': 6, '"bad"': 6, '"mobile"': 5, '"performance"': 5, '"product"': 4, '"redmi"': 3, '"money"': 3, '"display"': 3, '"great"': 2, '"awesome"': 2, '"flipkart"': 2, '"face"': 2, '"excellent"': 1, '"smile"': 1, '"note"': 1, '"quality"': 1, '"thank"': 1})


In [38]:
# Sắp xếp từ khóa theo tần suất giảm dần
sorted_keywords = keywords.most_common()

# Hiển thị từ khóa quan trọng nhất
print("Top 10 từ khóa quan trọng:")
for keyword, freq in sorted_keywords[:10]:
    print(f"{keyword}: {freq}")

Top 10 từ khóa quan trọng:
"phone": 10
"camera": 10
"i": 9
"good": 8
"battery": 8
"evil": 7
"price": 6
"bad": 6
"mobile": 5
"performance": 5


In [10]:
def encode_sentiment(star_rating):
    if star_rating >= 3.0:
        return 'Positive'
    else:
        return 'Negative'

df['Sentiment'] = df['Số sao đánh giá'].apply(encode_sentiment)